In [ ]:
# Imports
using JuMP, Plots, Gurobi, Statistics
include("../test/load.jl")
include("../test/tools/plots.jl")

In [ ]:
# Generating model
m = JuMP.Model(with_optimizer(Gurobi.Optimizer))
fn = :(x -> max(0, x[1] - 2 - 0.1*x[1]*x[2]^2, x[2] - 2))
@variable(m, x[1:2])
@variable(m, y)
gm = GlobalModel(model = m)
add_nonlinear_constraint(gm, fn, dependent_var = y)
set_param(gm.bbls[1], :n_samples, 1000)
bound!(gm, Dict(x[1] => [1, 5], x[2] => [1, 5]))

In [ ]:
uniform_sample_and_eval!(gm.bbls[1], lh_iterations=100)

In [ ]:
# Plotting data
plot_2d(gm.bbls[1], axes = [2,1])

In [ ]:
@time learn_constraint!(gm, hyperplane_config = (sparsity=1,))
gm.bbls[1].learners[end]

In [ ]:
# Plotting actual vs. predictions
using Statistics
using Plots
bbl = gm.bbls[1]
lnr = bbl.learners[end]
Ypredict = IAI.predict(lnr, bbl.X)
leaves = find_leaves(lnr)
leaf_index = IAI.apply(lnr, bbl.X)
display(Plots.plot(bbl.X[:,2], bbl.X[:,1], Ypredict,
     seriestype=:scatter, markersize = 3, zcolor=leaf_index, seriescolor=:phase, camera=(10,50), legend=false));
print("MSE: ", mean((Ypredict-bbl.Y).^2))

In [ ]:
# Plotting errors
display(Plots.plot(bbl.X[:,2], bbl.X[:,1], bbl.Y .- Ypredict,
     seriestype=:scatter, markersize = 3, zcolor=leaf_index, seriescolor=:phase, camera=(10,50), legend=false));